In [1]:
!git clone https://github.com/angel-one/smartapi-python.git

fatal: destination path 'smartapi-python' already exists and is not an empty directory.


In [2]:
!cd /content/smartapi-python

In [3]:
!pip install -r /content/smartapi-python/requirements_dev.txt

In [4]:
!pip install pyotp
!pip install logzero
!pip install websocket-client

In [5]:
!pip uninstall pycrypto
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.7 MB/s eta 0:00:00


In [6]:
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger

api_key = 'sP2JGPi3'
username = 'A61994896'
pwd = '3298'
smartApi = SmartConnect(api_key)
try:
    token = "FPELUPA7BHMESI3BPT34PAEPSY"
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

data = smartApi.generateSession(username, pwd, totp)

[I 241224 15:02:10 smartConnect:121] in pool


In [17]:
import pandas as pd
from datetime import datetime, timedelta

# Define the date range for the iteration
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 23)

# Initialize an empty list to store the results
results = []

# Loop through each day in the specified date range
current_date = start_date
while current_date <= end_date:
    # Define the date strings for the current day
    date_str = current_date.strftime("%Y-%m-%d")
    from_date = f"{date_str} 09:15"
    to_date = f"{date_str} 15:30"

    # Fetch the data for the current day
    historicParam = {
        "exchange": "NSE",
        "tradingsymbol": "BEL-EQ",
        "symboltoken": "383",
        "interval": "FIVE_MINUTE",
        "fromdate": from_date,
        "todate": to_date,
    }
    arr = smartApi.getCandleData(historicParam)
    data_array = arr['data']

    columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
    df = pd.DataFrame(data_array, columns=columns)

    # Convert datetime column to datetime object
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Filter for rows where the time is 9:40
    specific_time_data = df[df['datetime'].dt.time == pd.to_datetime("09:40:00").time()]

    if not specific_time_data.empty:
        high_price = specific_time_data['high'].iloc[0]
        low_price = specific_time_data['low'].iloc[0]
        stop_loss_diff = high_price - low_price

        f = 0
        subsequent_data = df[df['datetime'] > specific_time_data['datetime'].iloc[0]]
        buy_price, buy_time, sell_price, sell_time, profit_or_loss = None, None, None, None, 0

        for index, row in subsequent_data.iterrows():
            if row['high'] >= high_price and f == 0:
                buy_price = high_price
                sl_diff = high_price - low_price
                quantity = 1000 / sl_diff
                target_price = buy_price + (3 * sl_diff)
                buy_time = row['datetime']
                f = 1

            if f == 1:
                # Check for stop loss
                if row['low'] <= low_price:
                    sell_price = low_price
                    sell_time = row['datetime']
                    profit_or_loss = quantity * (sell_price - buy_price)
                    break

                # Check for target price
                if row['high'] >= target_price:
                    sell_price = target_price
                    sell_time = row['datetime']
                    profit_or_loss = quantity * (sell_price - buy_price)
                    break

        if f == 1 and sell_price:
            results.append({
                'date': date_str,
                'buy_time': buy_time,
                'buy_price': buy_price,
                'sell_time': sell_time,
                'sell_price': sell_price,
                'profit_or_loss': profit_or_loss,
            })
        else:
            results.append({
                'date': date_str,
                'buy_time': None,
                'buy_price': None,
                'sell_time': None,
                'sell_price': None,
                'profit_or_loss': 0,
            })

    else:
        results.append({
            'date': date_str,
            'buy_time': None,
            'buy_price': None,
            'sell_time': None,
            'sell_price': None,
            'profit_or_loss': 0,
        })

    # Move to the next day
    current_date += timedelta(days=1)

# Save the results to a CSV file
output_file = "daily_trading_results_01.csv"
results_df = pd.DataFrame(results)
results_df.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")


Results saved to daily_trading_results_01.csv


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Define the date range for the iteration
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 23)

# Initialize an empty list to store the results
results = []

# Loop through each day in the specified date range
current_date = start_date
while current_date <= end_date:
    # Define the date strings for the current day
    date_str = current_date.strftime("%Y-%m-%d")
    from_date = f"{date_str} 09:15"
    to_date = f"{date_str} 15:30"

    # Fetch the data for the current day
    historicParam = {
        "exchange": "NSE",
        "tradingsymbol": "BEL-EQ",
        "symboltoken": "383",
        "interval": "FIVE_MINUTE",
        "fromdate": from_date,
        "todate": to_date,
    }
    arr = smartApi.getCandleData(historicParam)
    data_array = arr['data']

    columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
    df = pd.DataFrame(data_array, columns=columns)

    # Convert datetime column to datetime object
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Filter for rows where the time is 9:40
    specific_time_data = df[df['datetime'].dt.time == pd.to_datetime("09:40:00").time()]

    if not specific_time_data.empty:
        high_price = specific_time_data['high'].iloc[0]
        low_price = specific_time_data['low'].iloc[0]
        stop_loss_diff = high_price - low_price

        f = 0
        subsequent_data = df[df['datetime'] > specific_time_data['datetime'].iloc[0]]
        buy_price, buy_time, sell_price, sell_time, profit_or_loss = None, None, None, None, 0

        for index, row in subsequent_data.iterrows():
            # Ensure we only check candles after the stock is purchased
            if row['high'] >= high_price and f == 0:
                buy_price = high_price
                sl_diff = high_price - low_price
                quantity = 1000 / sl_diff
                target_price = buy_price + (3 * sl_diff)
                buy_time = row['datetime']
                f = 1
                # Update subsequent_data to only include candles after the purchase
                subsequent_data = subsequent_data[subsequent_data['datetime'] > buy_time]

            if f == 1:
                # Check for stop loss
                if row['low'] <= low_price:
                    sell_price = low_price
                    sell_time = row['datetime']
                    profit_or_loss = quantity * (sell_price - buy_price)
                    break

                # Check for target price
                if row['high'] >= target_price:
                    sell_price = target_price
                    sell_time = row['datetime']
                    profit_or_loss = quantity * (sell_price - buy_price)
                    break

        if f == 1 and sell_price:
            results.append({
                'date': date_str,
                'buy_time': buy_time,
                'buy_price': buy_price,
                'sell_time': sell_time,
                'sell_price': sell_price,
                'profit_or_loss': profit_or_loss,
            })
        else:
            results.append({
                'date': date_str,
                'buy_time': None,
                'buy_price': None,
                'sell_time': None,
                'sell_price': None,
                'profit_or_loss': 0,
            })

    else:
        results.append({
            'date': date_str,
            'buy_time': None,
            'buy_price': None,
            'sell_time': None,
            'sell_price': None,
            'profit_or_loss': 0,
        })

    # Move to the next day
    current_date += timedelta(days=1)

# Save the results to a CSV file
output_file = "daily_trading_results_01.csv"
results_df = pd.DataFrame(results)
results_df.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")


In [18]:
import pandas as pd

# Load the uploaded CSV file
file_path = '/content/daily_trading_results_01.csv'
df = pd.read_csv(file_path)

# Calculate the total profit or loss from the CSV
total_profit_or_loss = df['profit_or_loss'].sum()

total_profit_or_loss


-90000.0

In [9]:
historicParam={
        "exchange": "NSE",
        "tradingsymbol": "BPCL-EQ",
        "symboltoken": "526",
        "interval": "FIVE_MINUTE",
        "fromdate": "2024-10-23 09:15",
        "todate": "2024-10-23 15:30"
        }
arr = smartApi.getCandleData(historicParam)
print(arr)

{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': [['2024-10-23T09:15:00+05:30', 321.7, 323.25, 318.5, 321.95, 512475], ['2024-10-23T09:20:00+05:30', 322.0, 322.85, 320.9, 320.9, 217777], ['2024-10-23T09:25:00+05:30', 321.05, 322.5, 319.55, 321.5, 170422], ['2024-10-23T09:30:00+05:30', 321.5, 321.85, 319.7, 320.25, 160248], ['2024-10-23T09:35:00+05:30', 320.25, 321.25, 318.1, 318.3, 156838], ['2024-10-23T09:40:00+05:30', 318.3, 318.8, 316.55, 316.6, 271421], ['2024-10-23T09:45:00+05:30', 316.8, 319.2, 316.6, 318.8, 232787], ['2024-10-23T09:50:00+05:30', 318.85, 320.2, 317.8, 319.7, 145106], ['2024-10-23T09:55:00+05:30', 319.7, 320.2, 319.2, 319.85, 103162], ['2024-10-23T10:00:00+05:30', 319.9, 321.75, 319.4, 321.3, 97662], ['2024-10-23T10:05:00+05:30', 321.3, 321.8, 320.5, 321.3, 70854], ['2024-10-23T10:10:00+05:30', 321.3, 322.05, 321.2, 321.85, 98134], ['2024-10-23T10:15:00+05:30', 321.85, 323.55, 321.7, 323.15, 175613], ['2024-10-23T10:20:00+05:30', 323.15, 323.25, 321

In [10]:
data_array = arr['data']
print(data_array)

[['2024-10-23T09:15:00+05:30', 321.7, 323.25, 318.5, 321.95, 512475], ['2024-10-23T09:20:00+05:30', 322.0, 322.85, 320.9, 320.9, 217777], ['2024-10-23T09:25:00+05:30', 321.05, 322.5, 319.55, 321.5, 170422], ['2024-10-23T09:30:00+05:30', 321.5, 321.85, 319.7, 320.25, 160248], ['2024-10-23T09:35:00+05:30', 320.25, 321.25, 318.1, 318.3, 156838], ['2024-10-23T09:40:00+05:30', 318.3, 318.8, 316.55, 316.6, 271421], ['2024-10-23T09:45:00+05:30', 316.8, 319.2, 316.6, 318.8, 232787], ['2024-10-23T09:50:00+05:30', 318.85, 320.2, 317.8, 319.7, 145106], ['2024-10-23T09:55:00+05:30', 319.7, 320.2, 319.2, 319.85, 103162], ['2024-10-23T10:00:00+05:30', 319.9, 321.75, 319.4, 321.3, 97662], ['2024-10-23T10:05:00+05:30', 321.3, 321.8, 320.5, 321.3, 70854], ['2024-10-23T10:10:00+05:30', 321.3, 322.05, 321.2, 321.85, 98134], ['2024-10-23T10:15:00+05:30', 321.85, 323.55, 321.7, 323.15, 175613], ['2024-10-23T10:20:00+05:30', 323.15, 323.25, 321.55, 321.7, 110096], ['2024-10-23T10:25:00+05:30', 321.8, 321.8,

In [11]:
# Step 1: Prepare the candles data

import datetime
from datetime import datetime
import pandas as pd


# data = []
# for candle in data_array:
#     if len(candle) == 6:
#         data.append({
#             'timestamp': candle[0],
#             'open': candle[1],
#             'high': candle[2],
#             'low': candle[3],
#             'close': candle[4],
#             'volume': candle[5]
#         })
#     else:
#         print(f"Invalid candle data: {candle}")

# Convert the data into a DataFrame
columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(data_array, columns=columns)

# Convert datetime column to the desired format
df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Save the DataFrame to a CSV file
output_file = "converted_data_01.csv"
df.to_csv(output_file, index=False)

# buy = False
# sell = False
# # Extracting candle at 9:40

# candle_at_9_40 = candles[5]
# print(candle_at_9_40)


# # Step 2: Implement the trading strategy
# def trading_strategy(candles):
#     trades = []  # Store trades in a list
#     buy = False
#     sell = False
#     i = 0  # Initialize index variable for main loop

#     # Step 3: Loop over the candles to find trades
#     while i < len(candles) - 2:
#         first_candle = candles[i]
#         second_candle = candles[i + 1]
#         third_candle = candles[i + 2]

#         # Check if second candle is inside first candle
#         if (second_candle['high'] < first_candle['high'] and
#             second_candle['low'] > first_candle['low']):

#             # Buy condition: Third candle crosses the high of second candle
#             if third_candle['high'] > second_candle['high']:
#                 trade = {
#                     'action': 'buy',
#                     'entry_price': second_candle['high'],  # Entry at second candle high
#                     'stop_loss': second_candle['low'],     # Stop loss at second candle low
#                     'timestamp': second_candle['timestamp']
#                 }
#                 bp = second_candle['high']
#                 sl = second_candle['low']
#                 ts = second_candle['timestamp']
#                 buy = True
#                 trades.append(trade)
#                 # print(f"Buy trade triggered at {third_candle['timestamp']}: entry {trade['entry_price']}, SL {trade['stop_loss']}")

#             if (buy):
#                 parsed_time = datetime.strptime(ts, '%Y-%m-%dT%H:%M:%S%z')
#                 formatted_time = parsed_time.strftime('%Y-%m-%d %H:%M')

#                 historicParam = {
#                     "exchange": "NSE",
#                     "tradingsymbol": "SBIN-EQ",
#                     "symboltoken": "3045",
#                     "interval": "ONE_MINUTE",
#                     "fromdate": formatted_time,
#                     "todate": "2024-10-28 15:30"
#                 }

#                 arr_one_min = smartApi.getCandleData(historicParam)
#                 data_array_one_min = arr_one_min['data']
#                 data_array_one_min = data_array_one_min[5:]  # Skip first 5 candles if necessary

#                 # Track the candle index where the one-minute loop ends
#                 for j, candle in enumerate(data_array_one_min):
#                     # Sell square-off condition
#                     if candle[2] >= ((bp - sl) * 2) + bp or candle[3] <= sl:
#                         trade = {
#                             'action': 'sell square off',
#                             'sell_price': candle[2] if candle[2] >= ((bp - sl) * 2) + bp else candle[3],
#                             'timestamp': candle[0]
#                         }
#                         trades.append(trade)
#                         buy = False
#                         i += j  # Move the main loop forward to where the one-minute loop ended
#                         break
#             elif third_candle['low'] < second_candle['low']:
#                 trade = {
#                     'action': 'sell',
#                     'entry_price': second_candle['low'],   # Entry at second candle low
#                     'stop_loss': second_candle['high'],    # Stop loss at second candle high
#                     'timestamp': second_candle['timestamp']
#                 }
#                 sp = second_candle['low']
#                 st_l = second_candle['high']
#                 sell = True
#                 ts = second_candle['timestamp']
#                 # print(ts)
#                 trades.append(trade)
#                 # print(f"Sell trade triggered at {third_candle['timestamp']}: entry {trade['entry_price']}, SL {trade['stop_loss']}")
#             if(sell):
#                 parsed_time = datetime.strptime(ts, '%Y-%m-%dT%H:%M:%S%z')
#                 formatted_time = parsed_time.strftime('%Y-%m-%d %H:%M')
#                 # print(formatted_time)
#                 historicParam={
#                       "exchange": "NSE",
#                       "tradingsymbol": "SBIN-EQ",
#                       "symboltoken": "3045",
#                       "interval": "ONE_MINUTE",
#                       "fromdate": formatted_time,
#                       "todate": "2024-10-28 15:30"
#                       }

#                 arr_one_min = smartApi.getCandleData(historicParam)
#                 data_array_one_min = arr_one_min['data']
#                 data_array_one_min = data_array_one_min[5:]
#                 # print(data_array_one_min)
#                 for j,candle in enumerate(data_array_one_min):
#                       # print(candle)
#                       # print(j)
#                       # print(f"Checking sell square-off: sp={sp}, st_l={st_l}, candle={candle}")

#                       if candle[3] <= sp - ((st_l - sp)*2) or candle[2] >= st_l:
#                           # print("inside")
#                           trade = {
#                           'action': 'buy  square off',
#                           'buy_price': candle[3] if candle[3] <= sp - ((st_l - sp)*2) else candle[2] ,  # Entry at second candle high    # Stop loss at second candle low
#                           'timestamp': candle[0]
#                       }
#                           trades.append(trade)
#                           sell = False
#                           i += j
#                           break
#         i += 1  # Increment the main loop index if no trade is made

#     return trades

# Step 3: Run the strategy
# trades = trading_strategy(candles)

# # Output all trades
# for trade in trades:
#     print(trade)


# # Step 4: AI-based Smart Position Sizing
# import numpy as np

# def ai_smart_allocation(current_balance, trade_risk):
#     """
#     Decide what fraction of the balance to allocate based on expected risk.
#     :param current_balance: The amount of money available.
#     :param trade_risk: Risk calculated from stop loss and volatility.
#     :return: The amount to allocate.
#     """
#     # Simple heuristic: Invest more when risk is low, less when risk is high.
#     risk_factor = max(0.01, min(0.05, 1 / trade_risk))  # Example: invest 1%-5% of capital based on risk
#     allocation = current_balance * risk_factor
#     return allocation

# def execute_trade(trade, balance):
#     trade_risk = abs(trade['entry_price'] - trade['stop_loss']) / trade['entry_price']
#     amount_to_invest = ai_smart_allocation(balance, trade_risk)
#     shares = amount_to_invest // trade['entry_price']
#     balance -= shares * trade['entry_price']

#     print(f"Trade executed: {trade['action']} {shares} shares at {trade['entry_price']} with SL {trade['stop_loss']}.")
#     return shares, balance, trade_risk

# # Step 5: Run the strategy with balance management and profit/loss calculation
# def calculate_profit_loss(trade, exit_price, shares):
#     if trade['action'] == 'buy':
#         return (exit_price - trade['entry_price']) * shares
#     elif trade['action'] == 'sell':
#         return (trade['entry_price'] - exit_price) * shares

# balance = 10000  # Starting balance
# initial_balance = balance
# trades = trading_strategy(candles)
# open_trades = []  # Store currently open trades

# for trade in trades:
#     shares, balance, trade_risk = execute_trade(trade, balance)
#     open_trades.append({
#         'trade': trade,
#         'shares': shares,
#         'trade_risk': trade_risk
#     })

#     # For simplicity, assuming every trade closes at the close price of the third candle
#     exit_price = candles[-1]['close']  # Using the last candle's close as exit for now (can be replaced with dynamic exit)

#     profit_loss = calculate_profit_loss(trade, exit_price, shares)
#     balance += profit_loss

#     print(f"Trade closed at {exit_price}. Profit/Loss: {profit_loss}. Current balance: {balance}")

# # Final balance and profit summary
# net_profit = balance - initial_balance
# print(f"Final Balance: {balance}")
# print(f"Net Profit: {net_profit}")


In [12]:
import pandas as pd

# Load the uploaded CSV file
file_path = '/content/converted_data_01.csv'  # Update this path if necessary
df = pd.read_csv(file_path)

# Convert the datetime column to a datetime object if it's not already
df['datetime'] = pd.to_datetime(df['datetime'])

# Initialize buy and sell status
buy = False
sell = False

# Filter for rows where the time is 9:40
specific_time_data = df[df['datetime'].dt.time == pd.to_datetime("09:40:00").time()]

# If there is data for 9:40, calculate the conditions for buying and selling
if not specific_time_data.empty:
    # Get the high and low prices at 9:40
    high_price = specific_time_data['high'].iloc[0]
    low_price = specific_time_data['low'].iloc[0]
    stop_loss_diff = high_price - low_price

    # Print buy details
    print(f"high price when the time is 9:40: {high_price}")
    print(f"Stop loss at 9:40: {low_price}")

    f = 0
    # Iterate through the candles after 9:40 to determine sell conditions
    subsequent_data = df[df['datetime'] > specific_time_data['datetime'].iloc[0]]
    print(subsequent_data)
    for index, row in subsequent_data.iterrows():

        if (row['high'] >= high_price and f==0):
          print(f"buy at: {row['datetime']}, at price: {row['high']}")
          buy_price = row['high']
          target_price = buy_price + (3 * (row['high'] - row['low']))
          buy = True
          f=1

        if (buy):
          # Condition 1: Loss - if buy price <= current candle's low price
          if row['low'] <= low_price:
              print(f"Sell at a loss at time: {row['datetime']}, price: {row['low']}")
              sell = True
              break

          # Condition 2: Profit - if target price <= any price (open, high, low, close)
          if (
              # target_price >= row['open']
              row['high'] >= target_price
              # or target_price >= row['low']
              # or target_price >= row['close']
          ):
              print(f"Sell at a profit at time: {row['datetime']}, price: {target_price}, high of candle at that time: {row['high']}")
              print(row['datetime'], row['open'], row['high'], row['low'], row['close'])
              sell = True
              break

    if not sell:
        print("No suitable price found for selling.")
else:
    print("No data available for 9:40.")








high price when the time is 9:40: 318.8
Stop loss at 9:40: 316.55
              datetime    open    high     low   close  volume
6  2024-10-23 09:45:00  316.80  319.20  316.60  318.80  232787
7  2024-10-23 09:50:00  318.85  320.20  317.80  319.70  145106
8  2024-10-23 09:55:00  319.70  320.20  319.20  319.85  103162
9  2024-10-23 10:00:00  319.90  321.75  319.40  321.30   97662
10 2024-10-23 10:05:00  321.30  321.80  320.50  321.30   70854
..                 ...     ...     ...     ...     ...     ...
70 2024-10-23 15:05:00  323.20  323.30  322.35  322.75   68166
71 2024-10-23 15:10:00  322.70  323.00  322.55  322.65   74931
72 2024-10-23 15:15:00  322.65  323.55  322.50  323.05  158248
73 2024-10-23 15:20:00  323.10  323.70  322.70  322.85  190236
74 2024-10-23 15:25:00  322.85  323.30  322.60  323.00  114886

[69 rows x 6 columns]
buy at: 2024-10-23 09:45:00, at price: 319.2
No suitable price found for selling.


In [13]:
import pandas as pd

# Load the uploaded CSV file
file_path = '/content/converted_data_01.csv'  # Update this path if necessary
df = pd.read_csv(file_path)

# Convert the datetime column to a datetime object if it's not already
df['datetime'] = pd.to_datetime(df['datetime'])

# Initialize buy and sell status
buy = False
sell = False
total_profit_or_loss = 0  # To track the total profit or loss

# Filter for rows where the time is 9:40
specific_time_data = df[df['datetime'].dt.time == pd.to_datetime("09:40:00").time()]

# If there is data for 9:40, calculate the conditions for buying and selling
if not specific_time_data.empty:
    # Get the high and low prices at 9:40
    high_price = specific_time_data['high'].iloc[0]
    low_price = specific_time_data['low'].iloc[0]
    stop_loss_diff = high_price - low_price

    # Print buy details
    print(f"High price when the time is 9:40: {high_price}")
    print(f"Stop loss at 9:40: {low_price}")

    f = 0
    # Iterate through the candles after 9:40 to determine sell conditions
    subsequent_data = df[df['datetime'] > specific_time_data['datetime'].iloc[0]]
    for index, row in subsequent_data.iterrows():
        if (row['high'] >= high_price and f == 0):
            # Buy at this price
            # buy_price = row['high']
            buy_price = high_price
            sl_diff = (high_price - low_price)
            print("sl diff = ", sl_diff)
            quantity = 1000 / (high_price - low_price)
            # target_price = buy_price + (3 * (row['high'] - row['low']))
            target_price = buy_price + (3 * sl_diff)
            print(f"Buy at: {row['datetime']}, at price: {buy_price}, quantity: {quantity}")
            buy = True
            f = 1

        if buy:
            # Condition 1: Loss - if buy price <= current candle's low price
            if row['low'] <= low_price:
                sell_price = row['low']
                profit_or_loss = quantity * (sell_price - buy_price)
                total_profit_or_loss += profit_or_loss
                print(f"Sell at a loss at time: {row['datetime']}, price: {sell_price}")
                print(f"Profit/Loss for this trade: {profit_or_loss}")
                sell = True
                break

            # Condition 2: Profit - if target price <= any price (open, high, low, close)
            if row['high'] >= target_price:
                sell_price = target_price
                profit_or_loss = quantity * (sell_price - buy_price)
                total_profit_or_loss += profit_or_loss
                print(f"Sell at a profit at time: {row['datetime']}, price: {sell_price}, high of candle at that time: {row['high']}")
                print(f"Profit/Loss for this trade: {profit_or_loss}")
                sell = True
                break

    if not sell:
        print("No suitable price found for selling.")
else:
    print("No data available for 9:40.")

# Print the total profit or loss
print(f"Total Profit or Loss: {total_profit_or_loss}")


High price when the time is 9:40: 318.8
Stop loss at 9:40: 316.55
sl diff =  2.25
Buy at: 2024-10-23 09:45:00, at price: 318.8, quantity: 444.44444444444446
Sell at a profit at time: 2024-10-23 13:35:00, price: 325.55, high of candle at that time: 325.85
Profit/Loss for this trade: 3000.0
Total Profit or Loss: 3000.0


In [14]:
# # Step 1: Define the initial amount and set up variables
# initial_balance = 10000
# balance = initial_balance
# position = None  # None means no open position
# entry_price = None
# trade_history = []  # To store the history of all trades

# # Step 2: Implement the strategy with profit/loss calculation
# def trading_strategy_with_profit(candles, initial_balance):
#     balance = initial_balance
#     units = 0  # Number of shares or units bought
#     position = None
#     entry_price = None
#     trade_history = []  # To store the history of all trades

#     # Loop over the candles to find trades
#     for i in range(len(candles) - 2):
#         first_candle = candles[i]
#         second_candle = candles[i + 1]
#         third_candle = candles[i + 2]

#         # Check if second candle is inside the first candle
#         if (second_candle['high'] < first_candle['high'] and
#             second_candle['low'] > first_candle['low']):

#             # Buy condition: Third candle crosses the high of second candle
#             if third_candle['high'] > second_candle['high']:
#                 if position is None:  # No open position, open a buy position
#                     entry_price = second_candle['high']
#                     units = balance // entry_price  # Max units we can buy
#                     balance -= units * entry_price  # Reduce balance by the amount spent
#                     position = 'long'  # We are now in a long (buy) position
#                     trade_info = {
#                         'action': 'buy',
#                         'timestamp': third_candle['timestamp'],
#                         'entry_price': entry_price,
#                         'units': units,
#                         'balance_after_trade': balance
#                     }
#                     trade_history.append(trade_info)
#                     print(f"Bought {units} units at {entry_price} on {third_candle['timestamp']}, remaining balance: {balance}")

#             # Sell condition: Third candle crosses the low of second candle
#             elif third_candle['low'] < second_candle['low']:
#                 if position == 'long':  # Close the buy position
#                     exit_price = second_candle['low']
#                     profit = units * (exit_price - entry_price)  # Calculate profit from the buy-sell
#                     balance += units * exit_price  # Add the amount gained back to balance
#                     position = None  # Reset position
#                     trade_info = {
#                         'action': 'sell',
#                         'timestamp': third_candle['timestamp'],
#                         'entry_price': exit_price,
#                         'units': units,
#                         'profit_loss': profit,
#                         'balance_after_trade': balance
#                     }
#                     trade_history.append(trade_info)
#                     print(f"Sold {units} units at {exit_price} on {third_candle['timestamp']}, profit/loss: {profit}, new balance: {balance}")
#                     units = 0  # Reset units

#     return balance, trade_history

# # Step 3: Run the strategy
# final_balance, trades = trading_strategy_with_profit(candles, initial_balance)

# # Output all trades and final balance
# print("\nTrade History:")
# for trade in trades:
#     if trade['action'] == 'buy':
#         print(f"Buy trade at {trade['timestamp']}: Bought {trade['units']} units at {trade['entry_price']}, balance after trade: {trade['balance_after_trade']}")
#     elif trade['action'] == 'sell':
#         print(f"Sell trade at {trade['timestamp']}: Sold {trade['units']} units at {trade['entry_price']}, Profit/Loss: {trade['profit_loss']}, balance after trade: {trade['balance_after_trade']}")

# print(f"\nInitial balance: {initial_balance}")
# print(f"Final balance: {final_balance}")
# print(f"Total profit/loss: {final_balance - initial_balance}")


In [15]:
# candles = []
# for candle in data_array:
#     if len(candle) == 6:  # Check that each candle has exactly 6 elements
#         candles.append({
#             'timestamp': candle[0],
#             'open': candle[1],
#             'high': candle[2],
#             'low': candle[3],
#             'close': candle[4],
#             'volume': candle[5]
#         })
#     else:
#         print(f"Invalid candle data: {candle}")

# # Step 2: Implement the strategy
# def trading_strategy(candles):
#     trades = []  # Store trades in a list

#     # Step 3: Loop over the candles to find trades
#     for i in range(len(candles) - 2):
#         first_candle = candles[i]
#         second_candle = candles[i + 1]
#         third_candle = candles[i + 2]

#         # Check if second candle is inside first candle
#         if (second_candle['high'] < first_candle['high'] and
#             second_candle['low'] > first_candle['low']):

#             # Buy condition: Third candle crosses the high of second candle
#             if third_candle['high'] > second_candle['high']:
#                 trade = {
#                     'action': 'buy',
#                     'entry_price': second_candle['high'],  # Entry at second candle high
#                     'stop_loss': second_candle['low'],     # Stop loss at second candle low
#                     'timestamp': third_candle['timestamp']
#                 }
#                 trades.append(trade)
#                 print(f"Buy trade triggered at {third_candle['timestamp']}: entry {trade['entry_price']}, SL {trade['stop_loss']}")

#             # Sell condition: Third candle crosses the low of second candle
#             elif third_candle['low'] < second_candle['low']:
#                 trade = {
#                     'action': 'sell',
#                     'entry_price': second_candle['low'],   # Entry at second candle low
#                     'stop_loss': second_candle['high'],    # Stop loss at second candle high
#                     'timestamp': third_candle['timestamp']
#                 }
#                 trades.append(trade)
#                 print(f"Sell trade triggered at {third_candle['timestamp']}: entry {trade['entry_price']}, SL {trade['stop_loss']}")

#     return trades

# # Step 4: Run the strategy
# trades = trading_strategy(candles)

# # Output all trades
# for trade in trades:
#     print(trade)

In [16]:
# # Step 1: Define the initial amount and set up variables
# initial_balance = 10000
# balance = initial_balance
# position = None  # None means no open position
# entry_price = None
# trade_history = []  # To store the history of all trades

# # Step 2: Simulate trading with the given trades
# def execute_trades(trades, initial_balance):
#     balance = initial_balance
#     units = 0  # Number of shares or units bought
#     position = None
#     entry_price = None
#     for trade in trades:
#         trade_info = {}  # Dictionary to hold trade info

#         if trade['action'] == 'buy':
#             if position is None:  # No open position, open a buy position
#                 entry_price = trade['entry_price']
#                 units = balance // entry_price  # Max units we can buy
#                 balance -= units * entry_price  # Reduce balance by the amount spent
#                 position = 'long'  # We are now in a long (buy) position
#                 trade_info = {
#                     'action': 'buy',
#                     'timestamp': trade['timestamp'],
#                     'entry_price': entry_price,
#                     'units': units,
#                     'balance_after_trade': balance
#                 }
#                 print(f"Bought {units} units at {entry_price} on {trade['timestamp']}, remaining balance: {balance}")

#         elif trade['action'] == 'sell':
#             if position == 'long':  # Close the buy position
#                 exit_price = trade['entry_price']
#                 profit = units * (exit_price - entry_price)  # Calculate profit from the buy-sell
#                 balance += units * exit_price  # Add the amount gained back to balance
#                 position = None  # Reset position
#                 trade_info = {
#                     'action': 'sell',
#                     'timestamp': trade['timestamp'],
#                     'entry_price': exit_price,
#                     'units': units,
#                     'profit_loss': profit,
#                     'balance_after_trade': balance
#                 }
#                 print(f"Sold {units} units at {exit_price} on {trade['timestamp']}, profit/loss: {profit}, new balance: {balance}")
#                 units = 0  # Reset units

#         if trade_info:  # If we made a trade, add it to the history
#             trade_history.append(trade_info)

#     return balance

# # Step 3: Run the strategy and calculate the final balance
# final_balance = execute_trades(trades, initial_balance)

# # Output the final result
# print("\nTrade History:")
# for trade in trade_history:
#     if trade['action'] == 'buy':
#         print(f"Buy trade at {trade['timestamp']}: Bought {trade['units']} units at {trade['entry_price']}, balance after trade: {trade['balance_after_trade']}")
#     elif trade['action'] == 'sell':
#         print(f"Sell trade at {trade['timestamp']}: Sold {trade['units']} units at {trade['entry_price']}, Profit/Loss: {trade['profit_loss']}, balance after trade: {trade['balance_after_trade']}")

# print(f"\nInitial balance: {initial_balance}")
# print(f"Final balance: {final_balance}")
# print(f"Total profit/loss: {final_balance - initial_balance}")
